In [2]:
from groq import Groq
import base64
import os 
# from dotenv import load_dotenv
import pyautogui
from io import BytesIO

In [ ]:
def encode_image(image_path):
  with open(image_path, "rb") as image_file:
    return base64.b64encode(image_file.read()).decode('utf-8')

def run_llm(prompt, base64_image, model="llama-3.2-11b-vision-preview"):
    client = Groq(api_key=os.environ["GROQ_API_KEY"])
    completion = client.chat.completions.create(
        model=model,
        messages=[
            {
                "role": "user",
                "content": [
                    {
                        "type": "text",
                        "text": f"{prompt}"
                    },
                    {
                        "type": "image_url",
                        "image_url": {
                            "url": f"data:image/jpeg;base64,{base64_image}",                    }
                    }
                ]
            }
        ],
        temperature=1,
        max_tokens=1024,
        top_p=1,
        stream=False,
        stop=None,
        response_format={"type": "json_object"},
    )

    return completion.choices[0].message.content

In [5]:
def take_screenshot(filename):
    # Take screenshot
    screenshot = pyautogui.screenshot()
    
    # Resize the image
    desired_size = (1024, 1024)
    resized_screenshot = screenshot.resize(desired_size)
    
    # Convert to RGB mode
    rgb_screenshot = resized_screenshot.convert('RGB')
    
    # Create a BytesIO object to store the image in memory
    buffer = BytesIO()
    
    # Save the image to the buffer in JPEG format with compression
    rgb_screenshot.save(buffer, format='JPEG', quality=50)
    
    # Get the bytes from the buffer and encode to base64
    base64_image = base64.b64encode(buffer.getvalue()).decode('utf-8')
    
    # Save the image to a file
    with open(filename, 'wb') as file:
        file.write(base64.b64decode(base64_image))
    
    # Close the buffer
    buffer.close()
    
    return base64_image

In [6]:
import time

for i in range(10):
    base64_image = take_screenshot(f"./data/screenshot_{i}.jpg")
    response = run_llm(prompt, base64_image)
    print(response)
    time.sleep(1)